## Q learning with the CartPole-v1

For a detailed description of the cartpole env, refer to [this](https://www.gymlibrary.dev/environments/classic_control/cart_pole/) site

In [48]:
!pip install gym[classic_control]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
import numpy as np
import gym
from tqdm import tqdm

### Configuring the display using matplotlib

In [50]:
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
from sklearn.preprocessing import KBinsDiscretizer
import math
import os
import sys
import pickle
from typing import Tuple

#### Creating some basic directories for storage

In [52]:
FOLDERS = ['rewards', 'graphs']

for folder in FOLDERS:
    if not os.path.exists(folder):
        os.makedirs(folder)

#### Define the policy
<!-- $\epsilon$ / $|A|$ + 1 - $\epsilon$ --- $max_{a}Q(s, a)$ -->

In [53]:
def policy(Q_table, state , eps = 0.05):
    """
    Epsilon greedy policy
    Choose the next action as follows
    eps / |A| + (1 - eps) --> greedy best
    eps / |A|             --> random
    """
    return np.argmax(Q_table[state]) if np.random.random() <= 1 - eps else np.random.randint(Q_table.shape[-1])

### The algorithm

In [54]:
# Helper function for rendering
env_rgb = lambda rendered_list: np.array(rendered_list).squeeze()

In [55]:
def Q_learning(environment, discretizer, Q_table, n_episodes, render = False, discount = 1, learning_rate = 0.95):
    """
    The implementation of the Q-Learning algorithm, here we take the discounts as 1 since all episodes are finite
    """
    prev = 0
    rewards = np.zeros(n_episodes)
    for ep in tqdm(range(n_episodes)):
        start_state = environment.reset()
        # print(start_state)
        discrete_state = discretizer.discretize(*start_state)

        if render:
            img = plt.imshow(env_rgb(environment.render()))
        
        done = False
        iters = 1
        episode_reward = 0
        while not done:
            # Step 1: Choose the first action using an e-greedy policy
            action = policy(Q_table, discrete_state, 1 / (ep + 1))
            new_state, reward, done, info, _ = environment.step(action)

            if render:
                plt.title(f"Episode no: {ep+1} Iteration no: {iters}")       
                img.set_data(env_rgb(environment.render()))
                display.display(plt.gcf())
                display.clear_output(wait=True)

            # Step 2: Now we greedily choose the next best action and find the cumulative reward
            new_discrete_state = discretizer.discretize(*new_state)
            next_action = np.argmax(Q_table[new_discrete_state])
            net_return = reward + discount * Q_table[new_discrete_state][next_action]
            episode_reward += reward
            if episode_reward >= 475:
                done = True

            # Step 3: Modify Q(s, a)
            Q_table[discrete_state][action] = (1 - learning_rate) * Q_table[discrete_state][action] + learning_rate * net_return
            
            # Step 4: Update state
            discrete_state = new_discrete_state
            state = new_state
            iters += 1

        rewards[ep] = iters - 1
        if prev < iters - 1:
            prev = iters - 1
#             print(f"Episode no: {ep+1} Iterations: {iters - 1}")

    return Q_table, rewards

### Testing

In [56]:
def play_episodes(environment, Q_res, n_episodes, discretizer, render = False):
    """
    Taking steps following the Q-function, i.e. from state s, take a that maximizes Q(s, a)
    """
    rewards = np.zeros(n_episodes)
    for episode in tqdm(range(n_episodes)):
        terminated = False
        state = environment.reset()
        state = discretizer.discretize(*state)
        if render:
            img = plt.imshow(env_rgb(environment.render()))

        episode_reward = 0
        while not terminated:
            # Select best action to perform in a current state
            action = np.argmax(Q_res[state])
            # Perform an action an observe how environment acted in response
            next_state, reward, terminated, info, _ = environment.step(action)
            
            episode_reward += reward
            
            if episode_reward >= 475:
                break
            
            if render:
                plt.title(f"Episode no: {episode+1} Reward: {episode_reward}")       
                img.set_data(env_rgb(environment.render()))
                display.display(plt.gcf())
                display.clear_output(wait=True)

            # Update current state
            next_state = discretizer.discretize(*next_state)
            state = next_state

        rewards[episode] = episode_reward

    return rewards

### Description

#### Action space

The action is an `ndarray` with shape `(1,)` which can take values {0, 1} indicating the direction of the fixed force the cart is pushed with.

#### State Space

The observation is an `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

| Num |      Observation      |         Min         |        Max        |   |
|:---:|:---------------------:|:-------------------:|:-----------------:|---|
| 0   | Cart Position         | -4.8                | 4.8               |   |
| 1   | Cart Velocity         | -Inf                | Inf               |   |
| 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |   |
| 3   | Pole Angular Velocity | -Inf                | Inf               |   |

#### Rewards

Since the goal is to keep the pole upright for as long as possible, a reward of `+1` for every step taken, including the termination step, is allotted. The threshold for rewards is `475` for v1.

#### Starting State

All observations are assigned a uniformly random value in (-0.05, 0.05)

#### Episode

The episode ends if any one of the following occurs:

- `Termination`: Pole Angle is greater than ±12°</li>
- `Termination`: Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)</li>
- `Truncation`: Episode length is greater than 500 (200 for v0)</li>


### Discretizer

In [57]:
class Discretizer:
    """
    The discretizer, important for handling the continuous state space,
    We utilize the sklearn library here
    """
    def __init__(self, n_bins, lower_bounds, upper_bounds):
        self.n_bins = n_bins
        self.discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
        self.discretizer.fit([lower_bounds, upper_bounds])
    
    def discretize(self, position, velocity, angle, angular_velocity):
        return tuple(map(int, self.discretizer.transform([[position, velocity, angle, angular_velocity]])[0]))

In [58]:
def analyze_and_plot(train_rewards, test_rewards, attr_name, attr_values, save_fig = False):
    """
    This basically takes in the rewards and stores it in a pickle file after plotting a graph.
    This is useful because it stores the data in case we need it for future analysis.
    """
    attr_values = list(map(str, attr_values))
    
    plt.cla()
    plt.title(f"Rewards from training with tweaks in {attr_name}")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    _ = plt.plot(np.arange(train_rewards.shape[-1]) + 1, train_rewards.T)
    plt.legend(attr_values)
    train_fig = plt.gca()
    plt.show()
    if save_fig:
        with open(f"rewards/train_rewards_{attr_name}.pkl", "wb") as f:
            pickle.dump(train_fig, f)
    plt.close()
    
    # Basically we print the correlations with the original value    
    train_correlations = np.corrcoef(train_rewards)
    print(f"Training time correlations: {train_correlations[2,:]}")
    
    mean_rewards = np.mean(test_rewards, axis=1)
    deviations = np.std(test_rewards, axis=1)
    print(f"Mean: {mean_rewards}, Standard deviation: {deviations}")
    
    plt.cla()
    _ = plt.plot(np.arange(test_rewards.shape[-1]) + 1, test_rewards.T)
    plt.title(f"Rewards from testing with tweaks in {attr_name}")
    plt.ylabel("Reward")
    plt.xlabel("New value")
    test_fig = plt.gca()
    plt.show()
    if save_fig:
        with open(f"rewards/test_rewards_{attr_name}.pkl", "wb") as f:
            pickle.dump(test_fig, f)
    plt.close()

In [59]:
def run_model(env, train_episodes, test_episodes, save_data = False):
    """
    The wrapper that calls the Q_learning function and returns rewards
    """
    print("Training model")
    Q_table = np.zeros((*N_BINS, env.action_space.n))
    Q_result, train_rewards = Q_learning(env, discretizer, Q_table, train_episodes, render=False)
    
    print("Testing model")
    test_rewards = play_episodes(env, Q_result, test_episodes, discretizer)
    print(f'Average reward over {test_episodes} episodes = {np.mean(test_rewards)} \n\n')
    
    return train_rewards, test_rewards

### The setup

In [60]:
# Number of partitions
N_BINS = (6, 2, 12, 12)

# Creation of environment
environment = gym.make('CartPole-v1', render_mode="rgb_array", new_step_api=True)

# Discretizer specifications
low_vals = environment.observation_space.low
high_vals = environment.observation_space.high

lower_bounds = [ low_vals[0], -3.5, low_vals[2], -3.5 ]
upper_bounds = [ high_vals[0], 3.5, high_vals[2], 3.5 ]

discretizer = Discretizer(N_BINS, lower_bounds, upper_bounds)

### The runner

In [ ]:
N_TRAIN_EPISODES = 50000
N_TEST_EPISODES = 1000

PLOT = True
SAVE = True

attributes = ["gravity", "masscart", "masspole", "length", "force_mag"]

for attr in attributes:
    print(f"Tweaking {attr}")
    print(f"Tweaking {attr}", file=sys.stderr)
    
    # obtain original value
    orig_value = getattr(environment, attr)
    
    all_train_rewards = np.zeros((5, N_TRAIN_EPISODES))
    all_test_rewards = np.zeros((5, N_TEST_EPISODES))
    
    new_values = np.linspace(0, 2 * orig_value, 5)
    new_values = np.round(new_values, decimals = 2)
    
    for idx, value in enumerate(new_values):
        setattr(environment, attr, value)
        train_rewards, test_rewards = run_model(environment, N_TRAIN_EPISODES, N_TEST_EPISODES, save_data = True) 
        print(f"Mean {np.mean(test_rewards)} and deviation {np.std(test_rewards)}")
        all_train_rewards[idx] = train_rewards
        all_test_rewards[idx] = test_rewards
    
    if PLOT:
        analyze_and_plot(all_train_rewards, all_test_rewards, attr, new_values, save_fig = SAVE)
        
    # reset to original value
    setattr(environment, attr, orig_value)

### Data Analysis
Here we retrieve data from the pickle files and carry out our analysis

In [ ]:
attributes = ["gravity", "masscart", "masspole", "length", "force_mag"]
WINDOW_SIZE = 2000

X = np.arange(0, N_TRAIN_EPISODES, WINDOW_SIZE)

def save_train(attr, train_rewards, new_values, X):
    plt.cla()
    print("Train")
    plt.title(f"Rewards from training with tweaks in {attr}")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    _ = plt.plot(X, np.array(train_rewards).T)
    _ = plt.legend(new_values)
    plt.savefig(f"graphs/train_rewards_{attr}.jpg")
    plt.show()
    plt.close()

def save_test(attr, new_values, mu):
    plt.cla()
    print("Test")
    plt.title(f"Mean rewards from testing with tweaks in {attr}")
    plt.xlabel(f"New values of {attr}")
    plt.ylabel("Mean reward")
    _ = plt.bar(new_values, mu)
    plt.savefig(f"graphs/test_rewards_{attr}.jpg")
    plt.show()
    plt.close()


for attr in attributes:
    with open(f"rewards/train_rewards_{attr}.pkl", "rb") as f:
        axes = pickle.load(f)

    orig_value = getattr(environment, attr)
    new_values = np.linspace(0, 2 * orig_value, 5)
    new_values = np.round(new_values, decimals = 2)
    new_values = list(map(str, new_values))
    
    train_rewards = []
    
    for line in axes.lines:
        my_rewards = np.array(line.get_ydata())
        train_rewards.append(np.convolve(my_rewards, np.ones(WINDOW_SIZE))[X] / WINDOW_SIZE)
        
    save_train(attr, train_rewards, new_values, X)   

    with open(f"rewards/test_rewards_{attr}.pkl", "rb") as f:
        axes = pickle.load(f)
    
    test_rewards = []
    for line in axes.lines:
        my_rewards = np.array(line.get_ydata())
        test_rewards.append(my_rewards)
    test_rewards = np.array(test_rewards)
    mu = np.mean(test_rewards, axis=1)

    save_test(attr, new_values, mu)